In [1]:
import numpy as np
import os
import pandas as pd
import scipy.interpolate
import scipy.signal
import csv
import re
import scipy.io
import pandas as pd
import scipy.signal as signal
from numpy.linalg import lstsq




In [2]:
#Selected channels to snr 
channels = ['ECG', 'DZ', 'DZDT', 'Z0', 'SBP', 'DBP', 'MBP', 'HR', 'SV', 'LVET','PI','MS', 'CO','TPR','TPRCGS', 'wr', 'tl', 'tr']

In [3]:

def get_snr2(csv_path, signal_name): 
    """
    Calculates the signal-to-noise ratio (SNR) for a given signal in a CSV file,
    using parabolic interpolation.

    Arguments:
    csv_path (str): Path to the CSV file containing the data.
    signal_name (str): Name of the column containing the signal for which SNR is to be calculated.

    Returns:
    float: Estimated signal-to-noise ratio (SNR) in decibels (dB).
    """
    df = pd.read_csv(csv_path)
    df = df.dropna(subset=[signal_name])
    sn = df[signal_name].values

    # Autocorrelation function
    acorrsn = signal.correlate(sn, sn, mode='full')
  
    # Interpolation and estimation
    N0 = len(sn)  # Number of signal samples
    ix4 = np.array([N0-3, N0-2, N0, N0+1])  # 4 real X coordinates of the autocorrelation samples
    ix = np.array([0, 1, 3, 4])  # Apparent X coordinates (to avoid bad conditioning in LS)
    r4 = acorrsn[ix4]  # 4 Y coordinates of the autocorrelation
    S = np.column_stack((ix**2, ix, np.ones(4)))  # Sensitivity matrix of the parabola

    # Estimate the parameters ABC of the parabola using least squares
    par, _, _, _ = lstsq(S, r4, rcond=None)

    Delta = par[1]**2 - 4 * par[0] * par[2]  # Determinant of the parabola
    r0int = -Delta / (4 * par[0])  # Parabolic interpolation: maximum = -B/2A
    r0 = acorrsn[N0-1]  # Middle value of the autocorrelation (lag 0)
    mu = np.mean(sn)  # Mean value of the signal
    SNRest = 10 * np.log10((r0int - mu**2) / (r0 - r0int))  # SNR estimate [dB]
    
    return SNRest



def calculate_snr(csv_path, signal_name):
     """
    Calculates the signal-to-noise ratio (SNR) for a given signal in a CSV file,
    using mean and standard deviation.

    Arguments:
    csv_path (str): Path to the CSV file containing the data.
    signal_name (str): Name of the column containing the signal for which SNR is to be calculated.

    Returns:
    float: Signal-to-noise ratio (SNR).
    
    Raises:
    ValueError: If the standard deviation of the signal is zero.
    """
    df = pd.read_csv(csv_path)
    df = df.dropna()
    

    # Picking the signal
    signal_column = df[signal_name]

    # OCalculating SNR
    mean_value = signal_column.mean()
    std_deviation = signal_column.std()

    if std_deviation == 0:
        raise ValueError("Standard deviation is equal to zero. SNR cannot be calculated.")

    snr = mean_value / std_deviation

    return snr

Pipeline for S1

In [38]:
root_path = '/home/ubuntu/eSportData/jupyter-data/VU_AMS/samples/snr'  # Set your path to data directory here
path = '/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output'

file_names = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
file_names = sorted(file_names)  # Adjust sorting method if needed

# Prepare the DataFrame to hold all results
columns = ['Filename'] + channels
results_df = pd.DataFrame(columns=columns)

# Process each file
for filename in file_names:

    file_path = os.path.join(path, filename)
    print(file_path)
    snr_values = {'Filename': filename}
    
    for signal_name in channels:
        try:
           
            snr = get_snr2(file_path, signal_name)
            snr_values[signal_name] = snr
        except:
            snr = -9999
            snr_values[signal_name] = snr
        
    # Convert the dictionary to a DataFrame and concatenate it with the results DataFrame
    snr_df = pd.DataFrame([snr_values])
    results_df = pd.concat([results_df, snr_df], ignore_index=True)

# Save the results to a CSV file
results_df.to_csv('snr_summary_trial_s1.csv', index=False)

/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p1.csv


/tmp/ipykernel_1561375/2091979379.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, snr_df], ignore_index=True)


/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p100.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p101.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p102.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p103.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p104.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p105.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p106.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p107.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p108.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p109.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p11.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p110.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p111.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p112.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p113.csv
/home/ubunt

/home/ubuntu/eSportData/venv/lib/python3.10/site-packages/scipy/signal/_signaltools.py:1055: RuntimeWarning: invalid value encountered in log
  fft_ops = 3 * N * np.log(N)  # 3 separate FFTs of size full_out_shape
/home/ubuntu/eSportData/venv/lib/python3.10/site-packages/scipy/signal/_signaltools.py:1055: RuntimeWarning: invalid value encountered in log
  fft_ops = 3 * N * np.log(N)  # 3 separate FFTs of size full_out_shape
/home/ubuntu/eSportData/venv/lib/python3.10/site-packages/scipy/signal/_signaltools.py:1055: RuntimeWarning: invalid value encountered in log
  fft_ops = 3 * N * np.log(N)  # 3 separate FFTs of size full_out_shape


/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p185.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p186.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p187.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p188.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p189.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p19.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p190.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p191.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p192.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p193.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p194.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p195.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p196.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p197.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p198.csv
/home/ubunt

...

/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p237.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p238.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p239.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p24.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p242.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p243.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p244.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p245.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p246.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p247.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p248.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p249.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p25.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p250.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p251.csv
/home/ubuntu

/home/ubuntu/eSportData/venv/lib/python3.10/site-packages/scipy/signal/_signaltools.py:1055: RuntimeWarning: invalid value encountered in log
  fft_ops = 3 * N * np.log(N)  # 3 separate FFTs of size full_out_shape
/home/ubuntu/eSportData/venv/lib/python3.10/site-packages/scipy/signal/_signaltools.py:1055: RuntimeWarning: invalid value encountered in log
  fft_ops = 3 * N * np.log(N)  # 3 separate FFTs of size full_out_shape
/home/ubuntu/eSportData/venv/lib/python3.10/site-packages/scipy/signal/_signaltools.py:1055: RuntimeWarning: invalid value encountered in log
  fft_ops = 3 * N * np.log(N)  # 3 separate FFTs of size full_out_shape


/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p287.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p288.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p289.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p29.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p290.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p291.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p292.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p293.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p294.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p295.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p296.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p297.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p298.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p299.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p3.csv
/home/ubuntu/

/tmp/ipykernel_1561375/2161600450.py:46: RuntimeWarning: invalid value encountered in scalar divide
  r0int = -Delta / (4 * par[0])  # Parabolic interpolation: maximum = -B/2A


/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p97.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p98.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p99.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/rejected_s1.txt


Pipeline for S2

In [ ]:
root_path = '/home/ubuntu/eSportData/jupyter-data/VU_AMS/samples/snr'  # Set your path to data directory here
path = '/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output' #Output path

file_names = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
file_names = sorted(file_names)  # Adjust sorting method if needed

# Prepare the DataFrame to hold all results
columns = ['Filename'] + channels
results_df = pd.DataFrame(columns=columns)

# Process each file
for filename in file_names:

    file_path = os.path.join(path, filename)
    print(file_path)
    snr_values = {'Filename': filename}
    
    for signal_name in channels:
        try:
           
            snr = get_snr2(file_path, signal_name)
           
            # except Exception as e:
            #     snr = None  # Handle exceptions such as missing columns or computation errors
            snr_values[signal_name] = snr
        except:
            snr = -9999
            snr_values[signal_name] = snr
        
    snr_df = pd.DataFrame([snr_values])
    results_df = pd.concat([results_df, snr_df], ignore_index=True)

# Save the results to a CSV file
results_df.to_csv('snr_summary_trial_s2.csv', index=False)

/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p1.csv


/tmp/ipykernel_2637241/2362165120.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, snr_df], ignore_index=True)


/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p10.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p100.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p101.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p102.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p103.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p104.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p105.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p106.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p107.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p108.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p109.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p11.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p110.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p111.csv
/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/S2_p112.csv
/home/ubuntu

Handling  exceptions

In [1]:
import pandas as pd

snr_summary_path = 'snr_summary_trial_s1.csv' #Loading a csv with calculated snr
df = pd.read_csv(snr_summary_path)

missing_data_rows = df[df.isnull().any(axis=1)] #Chcecking wchich ones are empty
missing_data_files = missing_data_rows['Filename'].tolist()
print(missing_data_files)

['S1_p102.csv', 'S1_p105.csv', 'S1_p113.csv', 'S1_p115.csv', 'S1_p116.csv', 'S1_p118.csv', 'S1_p125.csv', 'S1_p128.csv', 'S1_p129.csv', 'S1_p135.csv', 'S1_p137.csv', 'S1_p139.csv', 'S1_p142.csv', 'S1_p145.csv', 'S1_p154.csv', 'S1_p156.csv', 'S1_p160.csv', 'S1_p17.csv', 'S1_p170.csv', 'S1_p173.csv', 'S1_p175.csv', 'S1_p176.csv', 'S1_p178.csv', 'S1_p182.csv', 'S1_p187.csv', 'S1_p189.csv', 'S1_p19.csv', 'S1_p190.csv', 'S1_p192.csv', 'S1_p193.csv', 'S1_p194.csv', 'S1_p196.csv', 'S1_p197.csv', 'S1_p199.csv', 'S1_p2.csv', 'S1_p20.csv', 'S1_p202.csv', 'S1_p210.csv', 'S1_p217.csv', 'S1_p218.csv', 'S1_p222.csv', 'S1_p230.csv', 'S1_p231.csv', 'S1_p235.csv', 'S1_p236.csv', 'S1_p238.csv', 'S1_p243.csv', 'S1_p250.csv', 'S1_p267.csv', 'S1_p272.csv', 'S1_p273.csv', 'S1_p276.csv', 'S1_p28.csv', 'S1_p280.csv', 'S1_p285.csv', 'S1_p295.csv', 'S1_p34.csv', 'S1_p36.csv', 'S1_p38.csv', 'S1_p4.csv', 'S1_p42.csv', 'S1_p43.csv', 'S1_p46.csv', 'S1_p49.csv', 'S1_p61.csv', 'S1_p63.csv', 'S1_p64.csv', 'S1_p81.csv'

In [ ]:
data_path = '/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output'

for filename in missing_data_files:
    print(filename)
    file_path = os.path.join(data_path, filename)
    snr_values = {}

    for signal_name in channels:
        try: 
            snr = get_snr(file_path, 'ECG')
            snr_values[signal_name] = snr
        except: 
            snr = -9999 #Replacing nulls with -9999
            snr_values[signal_name] = snr
    

    # Update df with the new values
    for signal_name, snr in snr_values.items():
        df.loc[df['Filename'] == filename, signal_name] = snr


df.to_csv('snr_summary_s1_trial', index=False)